In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
import tensorflow_datasets as tfds
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [ ]:
(ds_train,ds_test), ds_info = tfds.load(
    'stanford_dogs',
    split =['train','test'],
    #shuffle_files=True,
    as_supervised=True, 
    with_info=True,  #it is used for ds_info 
)
# print(ds_info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

  0%|          | 0/12000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/stanford_dogs/0.2.0.incompleteVU1J0K/stanford_dogs-test.tfrecord


  0%|          | 0/8580 [00:00<?, ? examples/s]

Dataset stanford_dogs downloaded and prepared to /root/tensorflow_datasets/stanford_dogs/0.2.0. Subsequent calls will reuse this data.


In [ ]:
from PIL import Image as im

def resize(x):
  data = im.fromarray(x)
  data_new = data.resize((32,32))
  # data_new = tf.cast(data_new,tf.float32)
  #print(data_new.size)
  data_array = np.array(data_new)
  return data_array

train_images = []
train_labels = []
test_images =[]
test_labels=[]
for x,y in ds_train:
    train_images.append(resize(x.numpy()))
    train_labels.append(y.numpy())
for s,t in ds_test:
    test_images.append(resize(s.numpy()))
    test_labels.append(t.numpy())

train_images_arr = np.array(train_images)
train_labels_arr = np.array(train_labels)
test_images_arr = np.array(test_images)
test_labels_arr = np.array(test_labels)

train_images_arr = train_images_arr.astype('float32')
test_images_arr = test_images_arr.astype('float32')
train_images_arr = (train_images_arr - 127.5)/127.5
test_images_arr = (test_images_arr - 127.5)/127.5

BUFFER_SIZE = 300
BATCH_SIZE = 500
train_dataset = tf.data.Dataset.from_tensor_slices(train_images_arr).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


In [ ]:
# for x in train_dataset:
#   print(x.shape)


(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)
(500, 32, 32, 3)


In [ ]:

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
w_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
def deconv_block(inputs, num_filters, kernel_size, strides, bn=True):
    x = Conv2DTranspose(
        filters=num_filters,
        kernel_size=kernel_size,
        kernel_initializer=w_init,
        padding="same",
        strides=strides,
        use_bias=False
        )(inputs)

    if bn:
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)
    return x

def conv_block(inputs, num_filters, kernel_size, padding="same", strides=2, activation=True):
    x = Conv2D(
        filters=num_filters,
        kernel_size=kernel_size,
        kernel_initializer=w_init,
        padding=padding,
        strides=strides,
    )(inputs)

    if activation:
        x = LeakyReLU(alpha=0.2)(x)
        x = Dropout(0.3)(x)
    return x

latent_dim = 64
IMG_H = 32
IMG_W = 32 

def build_generator(latent_dim):
    f = [2**i for i in range(5)][::-1]
    filters = 32
    output_strides = 16
    h_output = IMG_H // output_strides
    w_output = IMG_W // output_strides

    noise = Input(shape=(latent_dim,), name="generator_noise_input")

    x = Dense(f[0] * filters * h_output * w_output, use_bias=False)(noise)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Reshape((h_output, w_output, 16 * filters))(x)
    print(x.shape)
    for i in range(1, 5):
        x = deconv_block(x,
            num_filters=f[i] * filters,
            kernel_size=5,
            strides=2,
            bn=True
        )
    x = conv_block(x,
        num_filters=3,  
        kernel_size=5,
        strides=1,
        activation=False
       )
    fake_output = Activation("tanh")(x)
     

    return Model(noise, fake_output, name="generator")
    

build_generator(latent_dim)


(None, 2, 2, 512)
